# Lesson 1: Advanced RAG Pipeline

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./bioengineering-2247773.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

30 

<class 'llama_index.schema.Document'>
Doc ID: 064feac3-d7ae-44e2-b744-5663a5a0b349
Text: Citation: Esmaeili, F.; Cassie, E.; Nguyen, H.P .T.; Plank,
N.O.V .; Unsworth, C.P .; Wang, A. Anomaly Detection for Sensor
Signals Utilizing Deep Learning Autoencoder-Based Neural Networks.
Bioengineering 2023 ,1, 0. https://doi.org/ Academic Editor: First
name Lastname Received: 13 February 2023 Revised: 7 March 2023
Accepted: 22 March 2023 Pu...


## Basic RAG pipeline

In [4]:
# Merge all of this content (= multiple document chunks) 
# into a single large document rather than having 
# one document per page which is the default set-up

from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
# Ingestion Pipeline

from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "What are performance metrics applied in this paper?"
)
print(str(response))

F1-score and accuracy were the performance metrics applied in this paper.


## Evaluation setup using TruLens

In [8]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What is the name of article?
How the writer used the AI method?
What is the contribution of this paper?
Who are the authors?
How many networks have been applied in this paper?
What deep learning methods has been applied?
What is the result of the paper?


In [9]:
# You can try your own question:
new_question = "What are the contributions of this paper?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

['What is the name of article?', 'How the writer used the AI method?', 'What is the contribution of this paper?', 'Who are the authors?', 'How many networks have been applied in this paper?', 'What deep learning methods has been applied?', 'What is the result of the paper?', 'What are the contributions of this paper?']


In [11]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [12]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2b59c07e0679ca198a0516c583a03434,"""What is the name of article?""","""An overview of deep learning based methods fo...",-,"{""record_id"": ""record_hash_2b59c07e0679ca198a0...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-05-16T01:17:06.156091"", ""...",2025-05-16T01:17:06.869123,1.0,0.2,0.300,[{'args': {'prompt': 'What is the name of arti...,[{'args': {'prompt': 'What is the name of arti...,[{'args': {'source': 'AC refers to the analyte...,0,2035,0.003063
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_ca7b081306336b34de5e12410343f443,"""How the writer used the AI method?""","""The writer utilized deep learning autoencoder...",-,"{""record_id"": ""record_hash_ca7b081306336b34de5...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-05-16T01:17:06.989329"", ""...",2025-05-16T01:17:08.375627,1.0,1.0,0.975,[{'args': {'prompt': 'How the writer used the ...,[{'args': {'prompt': 'How the writer used the ...,"[{'args': {'source': 'Citation: Esmaeili, F.; ...",1,2219,0.003386
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_0eefc81a3fb9f0e5a9a6ba01f0799959,"""What is the contribution of this paper?""","""The contribution of this paper lies in utiliz...",-,"{""record_id"": ""record_hash_0eefc81a3fb9f0e5a9a...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-05-16T01:17:08.487419"", ""...",2025-05-16T01:17:10.910671,1.0,1.0,NaN,[{'args': {'prompt': 'What is the contribution...,[{'args': {'prompt': 'What is the contribution...,NaN,2,2133,0.003252
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_bb98574d85b58c3dbd35879082e96521,"""Who are the authors?""","""The authors of the work are the experts who d...",-,"{""record_id"": ""record_hash_bb98574d85b58c3dbd3...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-05-16T01:17:11.028763"", ""...",2025-05-16T01:17:11.969069,1.0,NaN,NaN,"[{'args': {'prompt': 'Who are the authors?', '...",NaN,NaN,0,2092,0.003150
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_0083f775b13aad67a76262b34b760a00,"""How many networks have been applied in this p...","""Two networks have been applied in this paper.""",-,"{""record_id"": ""record_hash_0083f775b13aad67a76...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-05-16T01:17:12.091079"", ""...",2025-05-16T01:17:12.826520,1.0,NaN,NaN,[{'args': {'prompt': 'How many networks have b...,NaN,NaN,0,2070,0.003109


In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at https://s172-29-47-243p38560.lab-aws-production.deeplearning.ai/ .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [17]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [18]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [19]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [20]:
window_response = sentence_window_engine.query(
    "What type of threshold have been applied?"
)
print(str(window_response))

The threshold applied in the anomaly detection framework is based on the upper bound of the confidence interval.


In [21]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [22]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What is the name of article?
Anomaly Detection for Sensor Signals Utilizing Deep Learning Autoencoder-Based Neural Networks
How the writer used the AI method?
The writer utilized deep learning autoencoder-based neural networks for anomaly detection in sensor signals. Additionally, the integration of LSTM and vanilla autoencoders was employed to enhance the prediction models' effectiveness. The author also mentioned that combining different deep learning algorithms can improve the robustness, accuracy, and performance of prediction models.
What is the contribution of this paper?
The contribution of this paper is the utilization of deep learning autoencoder-based neural networks for anomaly detection in sensor signals.
Who are the authors?
The authors are Kwon, K.; Kwon, S.; Yeo, W.H.
How many networks have been applied in this paper?
Two networks have been applied in this paper.
What deep learning methods has been applied?
The deep learning methods applied in the study include autoencod

In [23]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,latency,total_cost
app_id,,,
Sentence Window Query Engine,1.0,1.0,0.000764


In [24]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-47-243p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [25]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [26]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [32]:
auto_merging_response = automerging_query_engine.query(
    "How do they build a network?"
)
print(str(auto_merging_response))

> Merging 4 nodes into parent node.
> Parent node id: 22a8e698-07a3-45b2-bbc6-b4c4d56a4f5b.
> Parent node text: Bioengineering 2023 ,1, 0 18 of 30
network and calculating the reconstruction errors, estimating ...

The network is built by designing an autoencoder-based network for anomaly detection.


In [28]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [29]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

> Merging 4 nodes into parent node.
> Parent node id: 327ec96f-6859-476b-8bee-85142f64bc02.
> Parent node text: Citation: Esmaeili, F.; Cassie, E.;
Nguyen, H.P .T.; Plank, N.O.V .;
Unsworth, C.P .; Wang, A. An...

> Merging 1 nodes into parent node.
> Parent node id: 86fb2933-5c6b-47ca-9faf-5a4a30fd01ad.
> Parent node text: Bioengineering 2023 ,1, 0 8 of 30
(a)
(b)
(c)
Figure 3. Cont.

> Merging 1 nodes into parent node.
> Parent node id: 084623cd-fa53-4d6e-8e39-f148d50b3a6c.
> Parent node text: Bioengineering 2023 ,1, 0 5 of 30
(a)
(b)
(c)
Figure 2. Cont.

> Merging 1 nodes into parent node.
> Parent node id: 1465cc13-b5f5-4484-811f-0f7b12e8648b.
> Parent node text: Bioengineering 2023 ,1, 0 8 of 30
(a)
(b)
(c)
Figure 3. Cont.

> Merging 1 nodes into parent node.
> Parent node id: 7ebedd72-ba41-45cf-8337-1143db9279b7.
> Parent node text: Bioengineering 2023 ,1, 0 5 of 30
(a)
(b)
(c)
Figure 2. Cont.

What is the name of article?
Anomaly Detection for Sensor Signals Utilizing Deep Learn

In [30]:
tru.get_leaderboard(app_ids=[])

,latency,total_cost
app_id,,
Automerging Query Engine,2.0,0.000826


In [31]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-47-243p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>